In [31]:
import pandas as pd

doc_dict = {}
id_dict = {}
word_count_dict = {}
country_set = set()

# pulls text IDs, country codes, document types, and word counts from the excel sheet,
# using it to divide the documents into a dictionary by ID
sources_df = pd.read_excel("./text/sampleSources.xlsx", sheet_name="texts")
for text_id, (country_code, doc_type), word_count in [(l[0], tuple(l[1].split()), l[2]) for l in sources_df[["textID", "country|genre", "# words"]].values.tolist()]:
    with open(f"./text/w_{country_code.lower()}_{doc_type.lower()}.txt", 'r',
              encoding="utf-8") as file:
        # add each text_id to id_dict
        if f"{country_code}_{doc_type}" not in id_dict:
            id_dict[f"{country_code}_{doc_type}"] = [text_id]
        else:
            id_dict[f"{country_code}_{doc_type}"].append(text_id)
        # makes country code set
        country_set.add(country_code)
        # finds correct text_id and adds every line in the document to the dictionary
        IS_DOC = False
        lines = file.readlines()
        for i, line in enumerate(lines):
            if line.strip().startswith(f"##{text_id}"):
                IS_DOC = True
            elif line.strip().startswith("##"):
                IS_DOC = False
            if IS_DOC:
                if text_id not in doc_dict:
                    doc_dict[text_id] = [w.lower() for w in line.split()]
                else:
                    doc_dict[text_id] += [w.lower() for w in line.split()]
        # adds word count to dictionary
        word_count_dict[text_id] = word_count

In [32]:
import copy
from collections import Counter

# make a counter for every word in the corpus
vocab = Counter({})
vocab['<UNK>'] = 0
for doc in doc_dict.values():
    for word in doc:
        if word in vocab:
            vocab[word] += 1
        else:
            vocab[word] = 1

# make a dictionary of sets for every country and record every word used by each country
# also make a word count for every country
vocab_sets = {country_code:set() for country_code in country_set}
country_word_counts = Counter({country_code:0 for country_code in country_set})
for country_code in country_set:
    for text_id in id_dict[f"{country_code}_B"] + id_dict[f"{country_code}_G"]:
        for word in doc_dict[text_id]:
            vocab_sets[country_code].add(word)
        country_word_counts[country_code] += word_count_dict[text_id]

# make new vocabulary sets, removing words that appear in less than
# 12.5%, 25%, 37.5%, and 50% of the countries datasets respectively
vocab_25 = vocab.copy()
vocab_50 = vocab.copy()
for word in vocab:
    COUNTRY_COUNT = 0
    for country_code in country_set:
        if word in vocab_sets[country_code]:
            COUNTRY_COUNT+=1
    if COUNTRY_COUNT / len(country_set) < 0.25:
        del vocab_25[word]
    if COUNTRY_COUNT / len(country_set) < 0.5:
        del vocab_50[word]

# Replace any words that appear in less than 25% of the countries’ datasets with the <UNK> token
doc_dict_25 = copy.deepcopy(doc_dict)
for text_id, doc in doc_dict_25.items():
    for i, word in enumerate(doc):
        if word not in vocab_25:
            doc_dict_25[text_id][i] = '<UNK>'
            vocab_25['<UNK>'] += 1

# Replace any words that appear in less than 50% of the countries’ datasets with the <UNK> token
doc_dict_50 = copy.deepcopy(doc_dict)
for text_id, doc in doc_dict_50.items():
    for i, word in enumerate(doc):
        if word not in vocab_50:
            doc_dict_50[text_id][i] = '<UNK>'
            vocab_50['<UNK>'] += 1

In [33]:
from sklearn.feature_extraction.text import CountVectorizer

# make a dataframe of one-hot representations of each text in each version of the dataset with their country labels
text_ids = []
texts = []
texts_25 = []
texts_50 = []
country_labels = []

for country_code in country_set:
    for text_id in id_dict[f"{country_code}_B"] + id_dict[f"{country_code}_G"]:
        text_ids.append(text_ids)
        texts.append(" ".join(doc_dict[text_id]))
        texts_25.append(" ".join(doc_dict_25[text_id]))
        texts_50.append(" ".join(doc_dict_50[text_id]))
        country_labels.append(country_code)

data = {
    'text_id': text_ids,
    'texts': texts,
    'texts_25': texts_25,
    'texts_50': texts_50,
    'country_labels': country_labels
}

df = pd.DataFrame(data)

vectorizer = CountVectorizer()
X = vectorizer.fit_transform(df['texts'])
X_25 = vectorizer.fit_transform(df['texts_25'])
X_50 = vectorizer.fit_transform(df['texts_50'])
y = df['country_labels']

In [34]:
def probability_breakdown(model, X_test, y_test):
    # makes a dictionary of probability Counters for every country label for each country's documents
    most_probable_country = {c:Counter({c:0 for c in country_set}) for c in country_set}
    for test_num, probabilities in enumerate(model.predict_proba(X_test)):
        for i, probability in enumerate(probabilities):
            most_probable_country[y_test.tolist()[test_num]][model.classes_[i]] += probability  

    return most_probable_country

In [35]:
def average_tries(model, X_test, y_test, print_output=False):
    """
    Takes the model and testing data.
    Returns average tries needed to return the right label,
    average tries per country, and number of tests per country.
    """
    
    # Matches each country label to its probability in a dataframe
    df = pd.DataFrame(model.predict_proba(X_test).tolist(), columns=model.classes_.tolist())

    # Sorts each probability distribution for highest probability
    # and records the number of iterations needed to get to the right label
    country_try_count = {c:0 for c in country_set}
    country_test_count = Counter(y_test)
    for i in range(len(df)):
        for try_count, (country_code, _) in enumerate(sorted(df.iloc[i].to_dict().items(), key=lambda item: item[1], reverse=True)):
            if country_code == y_test.tolist()[i]:
                country_try_count[country_code] += try_count+1
                break
    
    # Averages the country try count by number of tests
    country_try_count = {country:country_try_count[country]/country_test_count[country] for country in country_try_count}
    # Computes overal average try count
    avg_tries = sum(country_try_count.values())/len(country_try_count)
    
    # Either returns or prints the results
    if print_output:
        for country, try_count in sorted(country_try_count.items(), key=lambda item: item[1]):
            print(country, f"{try_count:.1f} tries", f"({country_test_count[country]} tests)")
        print("Average number of tries:", avg_tries)
    else:
        return avg_tries, country_try_count, country_test_count

In [36]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import numpy as np

def average_performance(model, X_dataset, random_states=10):
    """
    Takes the model, dataset, and number of random test splits to test.
    Prints average accuracy and tries between all test splits of the model.
    """
    accuracies = []
    tries = []
    avg_country_try_count = {c:0 for c in country_set}
    total_country_test_count = {c:0 for c in country_set}
    avg_most_probable_country = {c:Counter({c:0 for c in country_set}) for c in country_set}

    # trains models on the specified number of random splits
    for r in range(random_states):
        X_train, X_test, y_train, y_test = train_test_split(X_dataset, y, test_size=0.2, random_state=r)
        model.fit(X_train, y_train)
        # computes accuracies
        y_pred = model.predict(X_test)
        accuracies.append(accuracy_score(y_test, y_pred))
        # computes overall and country try counts
        avg_tries, country_try_count, country_test_count = average_tries(model, X_test, y_test)
        tries.append(avg_tries)
        # computes similar countries
        most_probable_country = probability_breakdown(model, X_test, y_test)
        for country in country_try_count:
            avg_country_try_count[country] += country_try_count[country]
            total_country_test_count[country] += country_test_count[country]
            avg_most_probable_country[country] += most_probable_country[country]
        print(".",end=" ")
    print()
    
    print("Average Accuracy:", np.mean(accuracies))
    print("Averaged Average tries:", np.mean(tries))
    # Sorts country try counts and prints them in order
    avg_country_try_count = {country:avg_country_try_count[country]/(random_states) for country in avg_country_try_count}
    for country, try_count in sorted(avg_country_try_count.items(), key=lambda item: item[1]):
        most_similar_five = " ".join([country for country, _ in avg_most_probable_country[country].most_common(5)])
        print(f"{country}: {try_count:4.1f} tries", f"({total_country_test_count[country]:3d} tests) | {most_similar_five}")

In [37]:
from sklearn.linear_model import LogisticRegression

# Logistic Regression model with class weights balanced
model = LogisticRegression(max_iter=2000, class_weight='balanced')
average_performance(model, X)
average_performance(model, X_25)
average_performance(model, X_50)

. . . . . . . . . . 
Average Accuracy: 0.3115151515151516
Averaged Average tries: 7.4150612910582625
GB:  2.5 tries (663 tests) | GB US AU CA IN
US:  2.7 tries (685 tests) | US GB AU CA NZ
AU:  4.4 tries (272 tests) | AU GB US CA NZ
IE:  5.1 tries (185 tests) | IE GB US PH CA
CA:  5.2 tries (226 tests) | US GB CA AU IE
NZ:  6.0 tries (121 tests) | NZ GB US AU PH
GH:  6.3 tries ( 64 tests) | GH GB US AU CA
IN:  6.3 tries (186 tests) | GB US IN CA NZ
PK:  6.4 tries ( 92 tests) | PK US GB AU IN
LK:  6.9 tries ( 82 tests) | LK GB US IN AU
KE:  7.4 tries ( 81 tests) | KE US GB JM CA
BD:  8.2 tries ( 64 tests) | BD GB US IE IN
ZA:  9.4 tries ( 81 tests) | US GB CA AU ZA
HK:  9.6 tries ( 74 tests) | US HK GB MY IN
TZ:  9.7 tries ( 54 tests) | GB US TZ IN IE
NG:  9.8 tries ( 77 tests) | GB US NG AU IN
SG:  9.9 tries ( 75 tests) | US GB SG IN MY
MY: 10.0 tries ( 76 tests) | GB US CA MY JM
JM: 11.3 tries ( 67 tests) | GB US IE JM AU
PH: 11.3 tries ( 75 tests) | GB US AU CA KE
. . . . . . . . . .

In [38]:
from sklearn.ensemble import RandomForestClassifier

# Random Forest Classifier with class weights balanced
rf_model = RandomForestClassifier(random_state=3, class_weight='balanced')
average_performance(rf_model, X)
average_performance(rf_model, X_25)
average_performance(rf_model, X_50)

. . . . . . . . . . 
Average Accuracy: 0.3242424242424242
Averaged Average tries: 6.511404179132822
GB:  1.4 tries (663 tests) | GB US CA AU IE
US:  1.6 tries (685 tests) | US GB CA AU IN
AU:  3.3 tries (272 tests) | GB US AU CA NZ
CA:  3.7 tries (226 tests) | US GB CA AU IE
IE:  3.7 tries (185 tests) | GB US IE AU CA
IN:  4.9 tries (186 tests) | GB US IN AU CA
LK:  5.1 tries ( 82 tests) | GB US LK IN CA
NZ:  5.2 tries (121 tests) | GB US NZ AU CA
GH:  6.4 tries ( 64 tests) | GB US GH AU CA
NG:  6.7 tries ( 77 tests) | GB US NG CA AU
BD:  6.9 tries ( 64 tests) | GB US BD IN CA
PK:  7.1 tries ( 92 tests) | GB US PK AU CA
SG:  8.3 tries ( 75 tests) | GB US AU SG CA
JM:  8.6 tries ( 67 tests) | GB US JM CA AU
KE:  8.6 tries ( 81 tests) | GB US AU CA KE
HK:  9.0 tries ( 74 tests) | GB US AU CA HK
TZ:  9.1 tries ( 54 tests) | GB US AU CA TZ
PH: 10.0 tries ( 75 tests) | GB US AU CA PH
MY: 10.1 tries ( 76 tests) | GB US AU CA MY
ZA: 10.5 tries ( 81 tests) | GB US AU CA ZA
. . . . . . . . . . 

In [39]:
from sklearn.neural_network import MLPClassifier

# Multi-layer Perceptron Model
mlp_model = MLPClassifier(random_state=3)
average_performance(mlp_model,X)
average_performance(mlp_model,X_25)
average_performance(mlp_model,X_50)

. . . . . . . . . . 
Average Accuracy: 0.3251515151515152
Averaged Average tries: 7.455500516530901
GB:  2.0 tries (663 tests) | GB US CA AU IN
US:  2.2 tries (685 tests) | US GB CA AU IN
CA:  3.7 tries (226 tests) | US GB CA AU IN
AU:  4.0 tries (272 tests) | GB US AU CA IE
IE:  5.4 tries (185 tests) | GB US IE CA AU
IN:  5.5 tries (186 tests) | GB US IN CA AU
NZ:  6.2 tries (121 tests) | GB US NZ CA AU
TZ:  6.6 tries ( 54 tests) | GB US AU CA TZ
PK:  7.1 tries ( 92 tests) | US GB PK CA AU
KE:  7.3 tries ( 81 tests) | US GB CA AU KE
LK:  7.7 tries ( 82 tests) | GB US LK AU CA
HK:  8.0 tries ( 74 tests) | US GB HK CA AU
PH:  8.4 tries ( 75 tests) | GB US CA PH AU
JM:  8.6 tries ( 67 tests) | GB US CA JM NZ
SG:  9.2 tries ( 75 tests) | GB US AU SG CA
BD:  9.2 tries ( 64 tests) | GB US BD AU CA
NG: 11.4 tries ( 77 tests) | GB US AU CA NG
GH: 11.6 tries ( 64 tests) | GB US AU GH CA
MY: 12.3 tries ( 76 tests) | US GB CA AU IN
ZA: 12.9 tries ( 81 tests) | US CA GB AU JM
. . . . . . . . . . 